In [1]:
!pip install ortools pandas ipywidgets fpdf


**FINAL CODE**

In [2]:
# Import necessary libraries
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, FileLink
from datetime import datetime, timedelta
from ortools.sat.python import cp_model
from fpdf import FPDF
import os

# Create the title and description
display(HTML("<h1>LJ University Exam Scheduler</h1>"))
display(HTML("<p>Create optimized exam schedules with automatic room assignments</p>"))

# Create input widgets
num_theory = widgets.IntSlider(value=3, min=1, max=10, step=1, description='Theory Exams:')
num_practical = widgets.IntSlider(value=2, min=0, max=10, step=1, description='Practical Exams:')
exam_type = widgets.Dropdown(options=['Regular', 'Remedial', 'Internal'], value='Regular', description='Exam Type:')
semester = widgets.Text(value='Semester 3', description='Semester:')
start_date_picker = widgets.DatePicker(description='Start Date:', value=datetime.now().date())

# Create expandable sections
theory_accordion = widgets.Accordion()
practical_accordion = widgets.Accordion()

# Function to update accordions when sliders change
def update_accordions(*args):
    # Create theory course inputs
    theory_inputs = [widgets.Text(description=f'Course {i+1}:', value=f'Theory Subject {i+1}')
                    for i in range(num_theory.value)]
    theory_box = widgets.VBox(theory_inputs)
    theory_accordion.children = [theory_box]
    theory_accordion.set_title(0, f'Theory Courses ({num_theory.value})')

    # Create practical course inputs
    practical_inputs = [widgets.Text(description=f'Course {i+1}:', value=f'Practical Subject {i+1}')
                       for i in range(num_practical.value)]
    practical_box = widgets.VBox(practical_inputs)
    practical_accordion.children = [practical_box]
    practical_accordion.set_title(0, f'Practical Courses ({num_practical.value})')

# Set up accordion change handlers
num_theory.observe(update_accordions, 'value')
num_practical.observe(update_accordions, 'value')

# Initialize accordions
update_accordions()

# Open accordions by default
theory_accordion.selected_index = 0
practical_accordion.selected_index = 0

# Output area for results and status messages
output = widgets.Output()

# Generate schedule button
generate_button = widgets.Button(
    description='Generate Schedule',
    button_style='success',
    icon='calendar'
)

# Create layout
main_layout = widgets.VBox([
    widgets.HBox([num_theory, num_practical]),
    widgets.HBox([exam_type, semester]),
    start_date_picker,
    theory_accordion,
    practical_accordion,
    generate_button,
    output
])

display(main_layout)

# Schedule generation function
def generate_schedule(b):
    with output:
        output.clear_output()
        print("Generating exam schedule...")

        # Collect theory courses
        theory_courses = []
        for i in range(num_theory.value):
            course_name = theory_accordion.children[0].children[i].value
            theory_courses.append((course_name, "Theory"))

        # Collect practical courses
        practical_courses = []
        for i in range(num_practical.value):
            if num_practical.value > 0 and i < len(practical_accordion.children[0].children):
                course_name = practical_accordion.children[0].children[i].value
                practical_courses.append((course_name, "Practical"))

        # Combine all courses
        courses = theory_courses + practical_courses

        if not courses:
            print("❌ Error: No courses entered")
            return

        # Get selected start date
        start_date = start_date_picker.value

        # Define constraints
        weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
        time_slots = {"Theory": "10:00 AM - 12:30 PM", "Practical": "10:00 AM - 01:00 PM"}
        classrooms = ["Room 101", "Room 207", "Room 208", "Room 307", "Room 209", "Room 308"]
        labs = ["Lab-7", "Lab-6", "Lab-8", "Lab-113", "Lab-205"]

        # Generate valid exam dates (excluding Sundays)
        exam_dates = []
        current_date = start_date
        while len(exam_dates) < len(courses):
            if current_date.weekday() != 6:  # Skip Sunday
                exam_dates.append(current_date)
            current_date += timedelta(days=1)

        # Create CP model
        model = cp_model.CpModel()
        exam_days = {c[0]: model.NewIntVar(0, len(courses)-1, c[0]) for c in courses}
        exam_classrooms = {
            c[0]: model.NewIntVar(0, len(classrooms)-1, f"{c[0]}_room") if c[1] == "Theory"
            else model.NewIntVar(0, len(labs)-1, f"{c[0]}_lab") for c in courses
        }

        # Add constraints
        model.AddAllDifferent(exam_days.values())
        model.AddAllDifferent(exam_classrooms.values())

        # Solve model
        solver = cp_model.CpSolver()
        status = solver.Solve(model)

        if status == cp_model.OPTIMAL:
            schedule = []
            for course, type_ in courses:
                day_idx = solver.Value(exam_days[course])
                room_idx = solver.Value(exam_classrooms[course])
                schedule.append([
                    exam_dates[day_idx].strftime("%d-%m-%Y"),
                    weekdays[exam_dates[day_idx].weekday()],
                    course,
                    type_,
                    time_slots[type_],
                    classrooms[room_idx] if type_ == "Theory" else labs[room_idx]
                ])

            # Sort by date
            schedule.sort(key=lambda x: datetime.strptime(x[0], "%d-%m-%Y"))

            # Create DataFrame
            df = pd.DataFrame(schedule, columns=["Date", "Day", "Course", "Type", "Time Slot", "Room/Lab"])

            # Display results
            print("\n✅ Schedule generated successfully!\n")
            display(df)

            # Save to Excel
            df.to_excel("exam_schedule.xlsx", index=False)

            # Generate PDF
            class PDF(FPDF):
                def header(self):
                    self.set_font("Arial", "B", 14)
                    self.cell(200, 10, "LJ UNIVERSITY SUMMER EXAM-2025", ln=True, align="C")
                    self.set_font("Arial", "B", 12)
                    self.cell(200, 10, f"{exam_type.value.upper()} EXAM - {semester.value.upper()}", ln=True, align="C")
                    self.ln(10)

                def create_table(self, data):
                    self.set_fill_color(200, 200, 200)
                    self.set_font("Arial", "B", 10)
                    col_widths = [30, 25, 40, 25, 40, 30]
                    headers = ["Date", "Day", "Course", "Type", "Time Slot", "Room/Lab"]
                    for i, header in enumerate(headers):
                        self.cell(col_widths[i], 10, header, 1, 0, "C", 1)
                    self.ln()
                    self.set_font("Arial", "", 10)
                    fill = 0
                    for row in data:
                        for i, item in enumerate(row):
                            self.cell(col_widths[i], 10, str(item), 1, 0, "C", fill)
                        self.ln()
                        fill = 1 - fill

            pdf = PDF()
            pdf.add_page()
            pdf.create_table(schedule)
            pdf.output("exam_schedule.pdf")

            # Provide download links
            print("\nDownload files:")
            display(FileLink('exam_schedule.xlsx', result_html_prefix="Excel Schedule: "))
            display(FileLink('exam_schedule.pdf', result_html_prefix="PDF Schedule: "))

        else:
            print("❌ No valid schedule could be found. Try changing your constraints.")

# Connect button to function
generate_button.on_click(generate_schedule)

load C:\Users\INSPIREBITS3\AppData\Local\Programs\Python\Python313\Lib\site-packages\ortools\.libs\zlib1.dll...
load C:\Users\INSPIREBITS3\AppData\Local\Programs\Python\Python313\Lib\site-packages\ortools\.libs\abseil_dll.dll...
load C:\Users\INSPIREBITS3\AppData\Local\Programs\Python\Python313\Lib\site-packages\ortools\.libs\utf8_validity.dll...
load C:\Users\INSPIREBITS3\AppData\Local\Programs\Python\Python313\Lib\site-packages\ortools\.libs\re2.dll...
load C:\Users\INSPIREBITS3\AppData\Local\Programs\Python\Python313\Lib\site-packages\ortools\.libs\libprotobuf.dll...
load C:\Users\INSPIREBITS3\AppData\Local\Programs\Python\Python313\Lib\site-packages\ortools\.libs\highs.dll...
load C:\Users\INSPIREBITS3\AppData\Local\Programs\Python\Python313\Lib\site-packages\ortools\.libs\ortools.dll...


In [ ]:
# Install required packages
!pip install ortools pandas ipywidgets fpdf matplotlib

# Import necessary libraries
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, FileLink, clear_output
from datetime import datetime, timedelta
from ortools.sat.python import cp_model
from fpdf import FPDF
import matplotlib.pyplot as plt
import io
import base64
import os

# Create styled header with CSS
display(HTML("""
<style>
.header {
    background-color: #003366;
    color: white;
    padding: 15px;
    border-radius: 10px;
    margin-bottom: 20px;
    text-align: center;
}
.subheader {
    color: #666;
    margin-bottom: 25px;
    text-align: center;
    font-style: italic;
}
.section-label {
    font-weight: bold;
    margin-top: 15px;
    color: #003366;
}
</style>
<div class="header">
    <h1>LJ University Exam Scheduler</h1>
</div>
<div class="subheader">
    Create optimized exam schedules with automatic room assignments
</div>
"""))

# Create tabbed interface
tabs = widgets.Tab()

# Tab 1: Basic Settings
basic_settings = widgets.VBox([
    widgets.HTML(value='<div class="section-label">General Exam Information</div>'),
    widgets.HBox([
        widgets.IntSlider(value=3, min=1, max=15, step=1, description='Theory Exams:',
                         style={'description_width': '120px'}, layout=widgets.Layout(width='50%')),
        widgets.IntSlider(value=2, min=0, max=10, step=1, description='Practical Exams:',
                         style={'description_width': '120px'}, layout=widgets.Layout(width='50%'))
    ]),
    widgets.HBox([
        widgets.Dropdown(options=['Regular', 'Remedial', 'Internal', 'Final'], value='Regular',
                        description='Exam Type:', style={'description_width': '120px'}, layout=widgets.Layout(width='50%')),
        widgets.Text(value='Semester 3', description='Semester:',
                    style={'description_width': '120px'}, layout=widgets.Layout(width='50%'))
    ]),
    widgets.DatePicker(description='Start Date:', value=datetime.now().date(),
                      style={'description_width': '120px'})
])

# Tab 2: Course Details
theory_accordion = widgets.Accordion()
practical_accordion = widgets.Accordion()
course_details = widgets.VBox([
    widgets.HTML(value='<div class="section-label">Enter Course Information</div>'),
    theory_accordion,
    practical_accordion
])

# Tab 3: Advanced Settings
advanced_settings = widgets.VBox([
    widgets.HTML(value='<div class="section-label">Room Assignment Settings</div>'),
    widgets.Textarea(
        value="Room 101\nRoom 207\nRoom 208\nRoom 307\nRoom 209\nRoom 308",
        description='Classrooms:',
        layout=widgets.Layout(width='80%', height='100px'),
        style={'description_width': '120px'}
    ),
    widgets.Textarea(
        value="Lab-7\nLab-6\nLab-8\nLab-113\nLab-205",
        description='Labs:',
        layout=widgets.Layout(width='80%', height='100px'),
        style={'description_width': '120px'}
    ),
    widgets.HTML(value='<div class="section-label">Time Slot Settings</div>'),
    widgets.Text(
        value="10:00 AM - 12:30 PM",
        description='Theory Time:',
        style={'description_width': '120px'},
        layout=widgets.Layout(width='50%')
    ),
    widgets.Text(
        value="10:00 AM - 01:00 PM",
        description='Practical Time:',
        style={'description_width': '120px'},
        layout=widgets.Layout(width='50%')
    ),
    widgets.Checkbox(
        value=True,
        description='Skip Sundays',
        style={'description_width': '120px'}
    ),
    widgets.Checkbox(
        value=False,
        description='Allow two exams per day',
        style={'description_width': '120px'}
    )
])

# Set up tabs
tabs.children = [basic_settings, course_details, advanced_settings]
tabs.set_title(0, 'General Settings')
tabs.set_title(1, 'Course Details')
tabs.set_title(2, 'Advanced Options')

# Function to update accordions when sliders change
def update_accordions(*args):
    # Get slider values
    num_theory_val = basic_settings.children[1].children[0].value
    num_practical_val = basic_settings.children[1].children[1].value

    # Create theory course inputs
    theory_inputs = []
    for i in range(num_theory_val):
        row = widgets.HBox([
            widgets.Text(
                value=f'Theory Subject {i+1}',
                description=f'Course {i+1}:',
                style={'description_width': '100px'},
                layout=widgets.Layout(width='70%')
            ),
            widgets.Dropdown(
                options=['Easy', 'Medium', 'Hard'],
                value='Medium',
                description='Difficulty:',
                style={'description_width': '100px'},
                layout=widgets.Layout(width='30%')
            )
        ])
        theory_inputs.append(row)

    theory_box = widgets.VBox(theory_inputs)
    theory_accordion.children = [theory_box]
    theory_accordion.set_title(0, f'Theory Courses ({num_theory_val})')

    # Create practical course inputs
    practical_inputs = []
    for i in range(num_practical_val):
        row = widgets.HBox([
            widgets.Text(
                value=f'Practical Subject {i+1}',
                description=f'Course {i+1}:',
                style={'description_width': '100px'},
                layout=widgets.Layout(width='70%')
            ),
            widgets.Dropdown(
                options=['Easy', 'Medium', 'Hard'],
                value='Medium',
                description='Difficulty:',
                style={'description_width': '100px'},
                layout=widgets.Layout(width='30%')
            )
        ])
        practical_inputs.append(row)

    practical_box = widgets.VBox(practical_inputs)
    practical_accordion.children = [practical_box]
    practical_accordion.set_title(0, f'Practical Courses ({num_practical_val})')

# Set up accordion change handlers
basic_settings.children[1].children[0].observe(update_accordions, 'value')
basic_settings.children[1].children[1].observe(update_accordions, 'value')

# Initialize accordions
update_accordions()

# Open accordions by default
theory_accordion.selected_index = 0
practical_accordion.selected_index = 0

# Output area for results and status messages
output = widgets.Output()

# Progress indicator
progress = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='info',
    style={'description_width': '100px'},
    orientation='horizontal'
)

# Generate schedule button with styling
generate_button = widgets.Button(
    description='Generate Schedule',
    button_style='success',
    icon='calendar',
    layout=widgets.Layout(width='200px', height='40px')
)

# Create final layout
main_layout = widgets.VBox([
    tabs,
    widgets.HBox([generate_button], layout=widgets.Layout(justify_content='center', margin='20px')),
    progress,
    output
])

display(main_layout)

# Helper function to create a visual calendar representation
def create_calendar_visualization(schedule_df):
    fig, ax = plt.subplots(figsize=(12, 8))

    # Get date range
    min_date = min(pd.to_datetime(schedule_df['Date'], format='%d-%m-%Y'))
    max_date = max(pd.to_datetime(schedule_df['Date'], format='%d-%m-%Y'))
    date_range = pd.date_range(start=min_date, end=max_date)

    # Create a dictionary to store exams by date
    exams_by_date = {}
    for _, row in schedule_df.iterrows():
        date_str = row['Date']
        date_obj = datetime.strptime(date_str, '%d-%m-%Y').date()
        if date_obj not in exams_by_date:
            exams_by_date[date_obj] = []
        exams_by_date[date_obj].append({
            'Course': row['Course'],
            'Type': row['Type'],
            'Room': row['Room/Lab']
        })

    # Set up the calendar grid
    ax.set_xlim(0, 7)
    num_weeks = (len(date_range) + 6) // 7
    ax.set_ylim(0, num_weeks)

    # Draw grid
    for i in range(8):
        ax.axvline(i, color='gray', linestyle='-', alpha=0.3)
    for i in range(num_weeks + 1):
        ax.axhline(i, color='gray', linestyle='-', alpha=0.3)

    # Label days
    days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    for i, day in enumerate(days):
        ax.text(i + 0.5, num_weeks + 0.05, day, ha='center', va='bottom')

    # Plot the exams
    for date in date_range:
        day_of_week = date.weekday()
        week_num = (date - min_date).days // 7
        y_pos = num_weeks - 1 - week_num

        # Add date number
        ax.text(day_of_week + 0.05, y_pos + 0.9, str(date.day), fontsize=10)

        # Add exams for this day
        if date.date() in exams_by_date:
            exams = exams_by_date[date.date()]
            for i, exam in enumerate(exams):
                y_offset = 0.7 - i * 0.2
                color = 'lightblue' if exam['Type'] == 'Theory' else 'lightgreen'
                ax.add_patch(plt.Rectangle((day_of_week + 0.1, y_pos + y_offset - 0.15), 0.8, 0.2,
                                         facecolor=color, alpha=0.8, edgecolor='gray'))
                ax.text(day_of_week + 0.5, y_pos + y_offset - 0.05, exam['Course'],
                      ha='center', va='center', fontsize=8)

    # Set title and labels
    plt.title('Exam Schedule Calendar', fontsize=14)
    plt.axis('off')

    # Convert plot to image
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    plt.close(fig)

    # Convert to base64 for embedding
    img_str = base64.b64encode(buf.read()).decode('utf-8')

    return f'<img src="data:image/png;base64,{img_str}" />'

# Schedule generation function
def generate_schedule(b):
    with output:
        clear_output()
        print("Generating exam schedule...")
        progress.value = 10

        # Get basic settings
        num_theory_val = basic_settings.children[1].children[0].value
        num_practical_val = basic_settings.children[1].children[1].value
        exam_type_val = basic_settings.children[2].children[0].value
        semester_val = basic_settings.children[2].children[1].value
        start_date_val = basic_settings.children[3].value

        # Get advanced settings
        classrooms_text = advanced_settings.children[1].value
        labs_text = advanced_settings.children[2].value
        theory_time = advanced_settings.children[4].value
        practical_time = advanced_settings.children[5].value
        skip_sundays = advanced_settings.children[6].value
        allow_two_per_day = advanced_settings.children[7].value

        # Parse rooms
        classrooms = [room.strip() for room in classrooms_text.strip().split('\n') if room.strip()]
        labs = [lab.strip() for lab in labs_text.strip().split('\n') if lab.strip()]

        progress.value = 20

        # Collect theory courses
        theory_courses = []
        if num_theory_val > 0 and len(theory_accordion.children) > 0:
            theory_box = theory_accordion.children[0]
            for i in range(num_theory_val):
                # Access the HBox that contains both the course name and difficulty widgets
                row = theory_box.children[i]
                course_name = row.children[0].value  # Text widget for course name
                difficulty = row.children[1].value   # Dropdown widget for difficulty
                theory_courses.append((course_name, "Theory", difficulty))

        # Collect practical courses
        practical_courses = []
        if num_practical_val > 0 and len(practical_accordion.children) > 0:
            practical_box = practical_accordion.children[0]
            for i in range(num_practical_val):
                # Access the HBox that contains both the course name and difficulty widgets
                row = practical_box.children[i]
                course_name = row.children[0].value  # Text widget for course name
                difficulty = row.children[1].value   # Dropdown widget for difficulty
                practical_courses.append((course_name, "Practical", difficulty))

        # Combine all courses
        courses = theory_courses + practical_courses

        if not courses:
            print("❌ Error: No courses entered")
            progress.value = 0
            return

        progress.value = 30

        # Define time slots
        time_slots = {"Theory": theory_time, "Practical": practical_time}
        weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

        # Generate valid exam dates
        exam_dates = []
        current_date = start_date_val
        while len(exam_dates) < (len(courses) * 2):  # Get more dates than needed
            if not (skip_sundays and current_date.weekday() == 6):  # Skip Sunday if configured
                exam_dates.append(current_date)
            current_date += timedelta(days=1)

        progress.value = 40

        try:
            # Create CP model
            model = cp_model.CpModel()

            # Variables
            exam_days = {}
            exam_rooms = {}

            for i, (course, type_, _) in enumerate(courses):
                # Day assignment variable
                exam_days[course] = model.NewIntVar(0, len(exam_dates)-1, f"{course}_day")

                # Room assignment variable
                if type_ == "Theory":
                    exam_rooms[course] = model.NewIntVar(0, len(classrooms)-1, f"{course}_room")
                else:  # Practical
                    exam_rooms[course] = model.NewIntVar(0, len(labs)-1, f"{course}_lab")

            progress.value = 50

            # Constraints

            # No course on same day unless allowed
            if not allow_two_per_day:
                model.AddAllDifferent(exam_days.values())

            # No room conflicts for same-type exams on same day
            for i, (course1, type1, _) in enumerate(courses):
                for j, (course2, type2, _) in enumerate(courses):
                    if i < j and type1 == type2:
                        # If same day, must have different rooms
                        same_day = model.NewBoolVar(f"same_day_{course1}_{course2}")
                        model.Add(exam_days[course1] == exam_days[course2]).OnlyEnforceIf(same_day)
                        model.Add(exam_days[course1] != exam_days[course2]).OnlyEnforceIf(same_day.Not())

                        model.Add(exam_rooms[course1] != exam_rooms[course2]).OnlyEnforceIf(same_day)

            # Difficulty spacing - harder exams should have more gap
            for i, (course1, _, diff1) in enumerate(courses):
                for j, (course2, _, diff2) in enumerate(courses):
                    if i < j:
                        if diff1 == "Hard" or diff2 == "Hard":
                            # At least 2 days between hard exams
                            model.Add(cp_model.LinearExpr.Abs(exam_days[course1] - exam_days[course2]) >= 2)

                        elif diff1 == "Medium" or diff2 == "Medium":
                            # At least 1 day between medium exams
                            model.Add(cp_model.LinearExpr.Abs(exam_days[course1] - exam_days[course2]) >= 1)

            progress.value = 60

            # Objective: Minimize the schedule duration
            max_day = model.NewIntVar(0, len(exam_dates)-1, "max_day")
            for course, _ in exam_days.items():
                model.Add(max_day >= exam_days[course])

            model.Minimize(max_day)

            # Solve the model
            progress.value = 70
            solver = cp_model.CpSolver()
            status = solver.Solve(model)

            if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
                progress.value = 80

                # Extract schedule
                schedule = []
                for course, course_type, difficulty in courses:
                    day_index = solver.Value(exam_days[course])
                    date = exam_dates[day_index]
                    date_str = date.strftime("%d-%m-%Y")

                    room_index = solver.Value(exam_rooms[course])
                    if course_type == "Theory":
                        room = classrooms[room_index]
                    else:
                        room = labs[room_index]

                    weekday = weekdays[date.weekday()]
                    time = time_slots[course_type]

                    schedule.append({
                        "Course": course,
                        "Type": course_type,
                        "Difficulty": difficulty,
                        "Date": date_str,
                        "Day": weekday,
                        "Time": time,
                        "Room/Lab": room
                    })

                # Create DataFrame and sort by date
                schedule_df = pd.DataFrame(schedule)
                schedule_df['Date_obj'] = pd.to_datetime(schedule_df['Date'], format='%d-%m-%Y')
                schedule_df = schedule_df.sort_values('Date_obj')
                schedule_df = schedule_df.drop('Date_obj', axis=1)

                progress.value = 90

                # Display results
                display(HTML("<h3>📅 Generated Exam Schedule</h3>"))
                display(schedule_df)

                # Create calendar visualization
                calendar_html = create_calendar_visualization(schedule_df)
                display(HTML("<h3>📊 Calendar View</h3>"))
                display(HTML(calendar_html))

                # Export to PDF
                current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
                pdf_filename = f"exam_schedule_{current_time}.pdf"

                class PDF(FPDF):
                    def header(self):
                        self.set_font('Arial', 'B', 15)
                        self.cell(0, 10, f'LJ University {exam_type_val} Exams - {semester_val}', 0, 1, 'C')
                        self.ln(10)

                    def footer(self):
                        self.set_y(-15)
                        self.set_font('Arial', 'I', 8)
                        self.cell(0, 10, f'Generated on {datetime.now().strftime("%Y-%m-%d %H:%M")}', 0, 0, 'C')

                pdf = PDF()
                pdf.add_page()

                # Add exam info
                pdf.set_font('Arial', 'B', 12)
                pdf.cell(0, 10, f"Exam Schedule", 0, 1)
                pdf.set_font('Arial', '', 10)

                # Table header
                col_widths = [60, 30, 40, 40, 30]
                headers = ['Course', 'Type', 'Date', 'Time', 'Room/Lab']
                for i, header in enumerate(headers):
                    pdf.cell(col_widths[i], 7, header, 1)
                pdf.ln()

                # Table data
                for _, row in schedule_df.iterrows():
                    pdf.cell(col_widths[0], 7, row['Course'], 1)
                    pdf.cell(col_widths[1], 7, row['Type'], 1)
                    pdf.cell(col_widths[2], 7, f"{row['Day']}, {row['Date']}", 1)
                    pdf.cell(col_widths[3], 7, row['Time'], 1)
                    pdf.cell(col_widths[4], 7, row['Room/Lab'], 1)
                    pdf.ln()

                # Save PDF
                pdf.output(pdf_filename)

                # Generate download link
                display(HTML("<h3>📄 Export Options</h3>"))
                display(FileLink(pdf_filename, result_html_prefix="Click here to download: "))

                # Also export to Excel
                excel_filename = f"exam_schedule_{current_time}.xlsx"
                schedule_df.to_excel(excel_filename, index=False)
                display(FileLink(excel_filename, result_html_prefix="Click here to download Excel: "))

                progress.value = 100
                display(HTML("<p style='color:green; font-weight:bold'>✅ Schedule generated successfully!</p>"))
            else:
                progress.value = 0
                display(HTML("<p style='color:red; font-weight:bold'>❌ No feasible schedule found. Try adjusting your constraints.</p>"))

                # Provide suggestions
                suggestions = [
                    "Increase the date range by starting earlier",
                    "Allow two exams per day",
                    "Add more rooms or labs",
                    "Reduce difficulty constraints",
                    "Decrease the number of exams"
                ]

                suggestion_html = "<ul>"
                for suggestion in suggestions:
                    suggestion_html += f"<li>{suggestion}</li>"
                suggestion_html += "</ul>"

                display(HTML(f"<p>Suggestions:</p>{suggestion_html}"))

        except Exception as e:
            progress.value = 0
            display(HTML(f"<p style='color:red; font-weight:bold'>❌ Error: {str(e)}</p>"))

# Add click handler for generate button
generate_button.on_click(generate_schedule)